# Component 10: Grad-CAM Explainability

Visualize what regions of the image the models focus on using Grad-CAM

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image
import os

OUTPUT_DIR = '../outputs/gradcam'
os.makedirs(OUTPUT_DIR, exist_ok=True)
print('✓ Setup complete')

✓ Setup complete


In [2]:

# Custom Label Smoothing Loss (Needed for loading models)
@tf.keras.utils.register_keras_serializable()
class LabelSmoothingLoss(tf.keras.losses.Loss):
    def __init__(self, num_classes=4, smoothing=0.1, **kwargs):
        super().__init__(**kwargs)
        self.num_classes = num_classes
        self.smoothing = smoothing
        
    def get_config(self):
        config = super().get_config()
        config.update({'num_classes': self.num_classes, 'smoothing': self.smoothing})
        return config
        
    def call(self, y_true, y_pred):
        y_true = tf.cast(y_true, tf.int32)
        y_true_one_hot = tf.one_hot(y_true, self.num_classes)
        y_true_smooth = y_true_one_hot * (1 - self.smoothing) + self.smoothing / self.num_classes
        return tf.keras.losses.categorical_crossentropy(y_true_smooth, y_pred)


## Grad-CAM Implementation

In [3]:
def find_last_conv_layer(model):
    """Find the last convolutional layer in the model."""
    for layer in reversed(model.layers):
        if isinstance(layer, tf.keras.layers.Conv2D):
            return layer.name
        # Check if it's a nested model
        if hasattr(layer, 'layers'):
            for sublayer in reversed(layer.layers):
                if isinstance(sublayer, tf.keras.layers.Conv2D):
                    return sublayer.name
    raise ValueError('No Conv2D layer found')

def generate_gradcam(img_array, model, conv_layer_name, pred_index=None):
    """Generate Grad-CAM heatmap."""
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(conv_layer_name).output, model.output]
    )
    
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]
    
    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    
    return heatmap.numpy()

print('✓ Grad-CAM functions defined')

✓ Grad-CAM functions defined


## Generate Heatmaps for All Models

In [4]:
test_df = pd.read_csv('../outputs/test_manifest.csv')
classes = sorted(test_df['class_name'].unique())
models = ['baseline_cnn', 'resnet50', 'resnet50_attention', 'efficientnetb0']

models_processed = 0
for model_name in models:
    model_path = f'../outputs/models/{model_name}_best.h5'
    if not os.path.exists(model_path):
        print(f'⚠️  Model not found: {model_path}')
        continue
    
    print(f'\nGenerating Grad-CAM for {model_name}...')
    model = tf.keras.models.load_model(model_path, custom_objects={'LabelSmoothingLoss': LabelSmoothingLoss, 'label_smoothing_loss': LabelSmoothingLoss})
    try:
        conv_layer = find_last_conv_layer(model)
        print(f'  Last conv layer: {conv_layer}')
    except ValueError as e:
        print(f'  ⚠️  {e}')
        continue
    
    model_dir = f'{OUTPUT_DIR}/{model_name}'
    os.makedirs(model_dir, exist_ok=True)
    
    # Generate for 5 samples per class
    for class_name in classes:
        class_samples = test_df[test_df['class_name'] == class_name].sample(
            n=min(5, len(test_df[test_df['class_name'] == class_name])),
            random_state=42
        )
        
        for idx, (_, row) in enumerate(class_samples.iterrows()):
            # Load image
            img = tf.keras.preprocessing.image.load_img(row['filepath'], target_size=(224, 224))
            img_array = tf.keras.preprocessing.image.img_to_array(img)
            img_array = np.expand_dims(img_array / 255.0, axis=0)
            
            # Generate heatmap
            heatmap = generate_gradcam(img_array, model, conv_layer)
            
            # Create overlay
            original = np.array(img)
            heatmap_resized = np.uint8(255 * heatmap)
            heatmap_resized = Image.fromarray(heatmap_resized).resize((original.shape[1], original.shape[0]))
            heatmap_resized = np.array(heatmap_resized)
            
            jet = cm.get_cmap('jet')
            jet_heatmap = jet(heatmap_resized)[:, :, :3]
            jet_heatmap = np.uint8(255 * jet_heatmap)
            overlay = np.uint8(jet_heatmap * 0.4 + original * 0.6)
            
            # Save visualization
            fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
            ax1.imshow(original)
            ax1.set_title('Original')
            ax1.axis('off')
            ax2.imshow(heatmap_resized, cmap='jet')
            ax2.set_title('Heatmap')
            ax2.axis('off')
            ax3.imshow(overlay)
            ax3.set_title('Overlay')
            ax3.axis('off')
            plt.suptitle(f'{class_name}', fontsize=12, fontweight='bold')
            plt.tight_layout()
            plt.savefig(f'{model_dir}/{class_name.replace(" ", "_")}_sample{idx}.png', dpi=150)
            plt.close()
    
    print(f'  ✓ Saved Grad-CAM visualizations to {model_dir}/')
    models_processed += 1

if models_processed > 0:
    print(f'\n✅ GRAD-CAM COMPLETE - Processed {models_processed} model(s)')
else:
    print('\n⚠️  NO MODELS FOUND - Please train models first (notebooks 05, 06, 06b, 07)')

⚠️  Model not found: ../outputs/models/baseline_cnn_best.h5
⚠️  Model not found: ../outputs/models/resnet50_best.h5
⚠️  Model not found: ../outputs/models/resnet50_attention_best.h5
⚠️  Model not found: ../outputs/models/efficientnetb0_best.h5

⚠️  NO MODELS FOUND - Please train models first (notebooks 05, 06, 06b, 07)
